In [1]:
import functools
import os
import re
import typing

import contextily
import geopandas
import pandas
import shapely.geometry
import tqdm.auto as tqdm
from matplotlib import colors, cm, pyplot
from mpl_toolkits.axes_grid1 import anchored_artists

# cache contextily tiles to disk - there will be a lot of data and we will be
# reusing a lot of it very often
contextily.tile.memory.store_backend.location = "CONTEXTILY_CACHE"

# progress bar for pandas.apply -> .progress_apply
tqdm.tqdm.pandas()

# typedefs
T_DataFrame = pandas.core.frame.DataFrame
T_Series = pandas.core.series.Series

# hacky way to typedef figure and axis; axis type cannot be declared directly
(figure, axis) = pyplot.subplots()
T_Figure = type(figure)
T_Axis = type(axis)
pyplot.close()

/home/leaf/miniconda3/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


# Generate tweets

In [2]:
# dict that maps 2019 BARI Geographical Infrastructure roads shapefile clusters
# to human-readable summaries
CLUSTER_MAPPING = {
    1: "residential",
    2: "residential",
    3: "commercial",
    4: "residential",
    5: "tax-exempt",
    6: "residential",
    7: "mixed-use commercial"
}

### Load data

In [4]:
def denormalize(normalized_lst: float) -> float:
    """ Denormalize LST data. """
    return 44 * normalized_lst + 72.4

# 2019 BARI Geographical Infrastructure roads shapefile
# * slightly older version than what is on Dataverse doi:10.7910/DVN/WOT075
gdf_roads = geopandas.read_file("input/shapefiles/roads_fin_clust_2019_11142019.shp")\
    .to_crs({"init": "epsg:3857"}) # to simplify plotting

# 2019 BARI Geographical Infrastructure roads attributes doi:10.7910/DVN/WOT075
df_roads = pandas.read_csv("input/Roads.2019.csv")

# 2019 BARI Geographical Infrastructure land parcels doi:10.7910/DVN/WOT075
df_parcels = pandas.read_csv("input/LandParcels.2019.csv")

# 2010 Boston Neighborhood Survey doi:10.7910/DVN/SE2CIX
# * has neighborhood sections  in `NBHDS89_` column
gdf_bns = geopandas.read_file("input/shapefiles/2010_BNS_by_NBHDS89_Shape.shp")\
    .to_crs(gdf_roads.crs) # to match gdf_roads

# 2010 BostonGIS Neighborhood Boundaries
# https://bostonopendata-boston.opendata.arcgis.com/datasets/3525b0ee6e6b427f9aab5d0a1d0a1a28_0
# * has neighborhood names in `Name` column
gdf_neighborhoods = geopandas.read_file("input/shapefiles/Boston_Neighborhoods.shp")\
    .to_crs(gdf_roads.crs) # to match gdf_roads

# Excerpt of modeled street segment heat data from O'Brien et al. doi:10.2105/AJPH.2020.305636
df_urbanheat = pandas.read_csv("input/urbanheat_for_models_trimmed.csv")
for lst_column in ["LST_CT", "LST_weighted"]:
    df_urbanheat[lst_column] = df_urbanheat[lst_column].apply(denormalize)

### Merge data

In [5]:
# each dataframe is iteratively left merged with columns from the right, after
# the right dataframe is deduplicated against columns from the left
gdf_merged = geopandas.GeoDataFrame(functools.reduce(
    lambda left, right: pandas.merge(
        left[left.columns.difference(right.columns).to_list() + ["TLID"]],
        right,
        how="left",
        on="TLID"
    ),
    [df_urbanheat, df_roads, gdf_roads]
))
gdf_merged.crs = gdf_roads.crs

gdf_roads_nodata = gdf_roads[~gdf_roads["TLID"].isin(gdf_merged["TLID"])]

### Functions to extract address ranges from a list of full addresses

In [6]:
def mostly_a_number(string: str) -> bool:
    """ Return True if a string is mostly composed of integer characters. """
    return sum(1 for char in string if char.isdigit()) > 0.5

def natural_keys(string: str) -> typing.List[typing.Union[str, int]]:
    """ Break up a string into a list of contiguous numbers and strings, in
    such a way that a list comparison can perform natural sorts.
    
    Credit: https://stackoverflow.com/a/5967539
    """
    return [
        int(char)
        if char.isdigit()
        else char
        for char in re.split(r"(\d+)", string)
    ]

def address_numbers(full_address: str) -> typing.List[str]:
    """ Extract strings corresponding to different house numbers in an address.
    
    Here, we define house numbers as strings consisting mostly of numbers. In
    this way, we are able to capture non-integer numbers like "10A", "5E", etc.
    """
    return [
        address_part
        # we drop the zipcode here by taking only indices [:-1]
        for address_part in full_address.replace("-", " ").split()[:-1]
        if mostly_a_number(address_part)
    ]

def address_range(addresses: typing.List[str]) -> str:
    """ Extract the address range for a given list of addresses. """
    numbers = sorted(
        set(
            address_number
            for address in addresses
            for address_number in address_numbers(address)
        ),
        key=natural_keys
    )
    if len(numbers) == 0:
        return ""
    elif len(numbers) == 1:
        return numbers[0]
    else:
        return "{}-{}".format(numbers[0], numbers[-1])

def address_range_from_tlid(tlid: int) -> str:
    """ Given a TLID, extract the address range. """
    return address_range(df_parcels[df_parcels["TLID"] == tlid]["full_address"])

### Functions to generate tweet text

In [7]:
def get_neighborhoods(geometry) -> typing.List[str]:
    """ Get a list of neighborhoods intersecting a certain spatial object.
    
    The BNS definition is preferred, but we revert to the Boston Neighborhoods
    definition if any sections unnamed in the BNS survey are encoutered. """
    
    sections = gdf_bns[gdf_bns.intersects(geometry)]["NBHDS89_"].to_list()
    if "Unnamed" in sections:
        return gdf_neighborhoods[
            gdf_neighborhoods.intersects(street["geometry"])
        ]["Name"].to_list()
    else:
        return sections

def generate_tweet_text(street: T_Series) -> str:
    """ Given a row from the streets dataframe, return tweet text. """
    
    land_surface_temp = street["LST_weighted"]
    
    mean_land_surface_temp = gdf_merged["LST_weighted"].mean()
    if land_surface_temp > mean_land_surface_temp:
        comparison_to_mean = "{:.2f}°F hotter than average"\
            .format(land_surface_temp - mean_land_surface_temp)
    else:
        comparison_to_mean = "{:.2f}°F cooler than average"\
            .format(mean_land_surface_temp - land_surface_temp)
    
    n_parcels = (df_parcels["TLID"] == street["TLID"]).sum()
    if n_parcels == 0:
        house_numbers = "This segment of"
    else:
        house_numbers = address_range_from_tlid(street["TLID"])
    
    neighborhoods = get_neighborhoods(street["geometry"])
    if len(neighborhoods) == 0:
        location = "in Boston"
    elif len(neighborhoods) > 1:
        location = "that stretches from {} to {}".format(*neighborhoods)
    else:
        location = "located in the {} neighborhood".format(neighborhoods[0])
        
    if street["Main"] == 1:
        street_type = "main street"
    else:
        street_type = "street"
    
    street_name = street["FULLNAM"]
    if street_name is None:
        street_name = "This"
    
    zoning = ""
    cluster = street["Cluster"]
    if n_parcels > 0:
        zoning_str = CLUSTER_MAPPING[cluster]
        if n_parcels > 1:
            zoning = ", mostly {}".format(zoning_str)
        else:
            zoning = " which is {}".format(zoning_str)
    
    return " ".join(
        """
            {house_numbers} {street_name} is a {street_type} {location}. Its
            average summer land surface temperature for 2002-2008 was
            {land_surface_temp:.1f}°F, {comparison_to_mean}. It has {n_parcels}
            parcel{parcels_plural}{zoning}.
        """.format(
            house_numbers=house_numbers,
            street_name=street_name,
            street_type=street_type,
            location=location,
            land_surface_temp=land_surface_temp,
            comparison_to_mean=comparison_to_mean,
            n_parcels=n_parcels,
            parcels_plural=(n_parcels != 1 and "s" or ""),
            zoning=zoning
        ).split()
    )

### Validation of tweet generating functions - ensure no errors

In [8]:
tweet_size_limit = 280
sample_size = 10

print("note: twitter character limit is {} (image not counted). sample tweets:".format(
    tweet_size_limit
))

# preview
for (index, street) in gdf_merged.sample(sample_size).iterrows():
    tweet_str = generate_tweet_text(street)
    print("\n{} characters{}\n{}".format(
        len(tweet_str),
        len(tweet_str) > tweet_size_limit and " (!)" or "",
        tweet_str
    ))

# validate all streets
for (index, street) in tqdm.tqdm(gdf_merged.iterrows(), total=len(gdf_merged), desc="validating tweets"):
    tweet_str = generate_tweet_text(street)
    if len(tweet_str) > tweet_size_limit:
        print(result)
        raise Exception("tweet too long")

note: twitter character limit is 280 (image not counted). sample tweets:

211 characters
65-71 Woodrow Ave is a street that stretches from Franklin Field to Norfolk. Its average summer land surface temperature for 2002-2008 was 88.8°F, 6.74°F cooler than average. It has 4 parcels, mostly commercial.

189 characters
This segment of This is a street located in the Corey Hill neighborhood. Its average summer land surface temperature for 2002-2008 was 102.5°F, 6.95°F hotter than average. It has 0 parcels.

207 characters
51-76 Tyler St is a street located in the Fairmount Hill neighborhood. Its average summer land surface temperature for 2002-2008 was 87.6°F, 7.99°F cooler than average. It has 12 parcels, mostly residential.

207 characters
1-30 Cedarcrest Cir is a street located in the The Grove neighborhood. Its average summer land surface temperature for 2002-2008 was 88.8°F, 6.78°F cooler than average. It has 16 parcels, mostly residential.

209 characters
11-37 Gerald Rd is a street l

### Write out generated tweets

In [11]:
df_tweets = pandas.merge(
    gdf_merged["TLID"],
    gdf_merged.progress_apply(generate_tweet_text, axis=1).rename("tweet"),
    left_index=True,
    right_index=True
)
df_tweets.sample(frac=1).to_csv("tweets.csv", index=False) # shuffle and write
df_tweets.sample(5)

,TLID,tweet
8117,85706092,235-252 Gallivan Blvd is a main street located...
875,85695228,116-122 Warren St is a street located in the C...
2431,85697208,411-417 Bunker Hill St is a main street locate...
8403,85720624,151-164 Hebron St is a street located in the M...
7165,85720381,1/3/2005-45 Wilbert Rd is a street located in ...


# Generate images (draft)
This code is copied into a standalone script in order to run quickly on a cluster and also avoid memory leaks.

In [7]:
def make_square(axis):
    """ Force an axis to be square by having the smaller dimension equal the
    size of the larger dimension.
    
    Changing the aspect ratio can distort basemaps; this method does not.
    """
    xlim = axis.get_xlim()
    ylim = axis.get_ylim()
    width = xlim[1] - xlim[0]
    height = ylim[1] - ylim[0]
    if width > height: # make taller
        center_y = (ylim[0] + ylim[1])/2
        axis.set_ylim([
            center_y - width/2,
            center_y + width/2
        ])
    else: # make wider
        center_x = (xlim[0] + xlim[1])/2
        axis.set_xlim([
            center_x - height/2,
            center_x + height/2
        ])

In [13]:
column = "LST_weighted"
margin = 2.5 # times the size of the original figure
basemap = contextily.providers.CartoDB.Positron
inset_basemap = contextily.providers.CartoDB.PositronNoLabels
inset_linewidth = 5
inset_position = [0.55, 0.05, 0.4, 0.6] # x y w h

def generate_image(street: T_Series) -> typing.Tuple[T_Figure, T_Axis]:
    # forcing the aspect ratio slightly resizes the inset. i'm not sure how to
    # get the exact figure positions of the inset after resize, so a correction is
    # manually determined by trial and error and then applied.
    inset_adjust = 0.01

    # initialize colors
    colormap = cm.get_cmap("magma", 12)
    normalize = colors.Normalize(
        vmin=gdf_merged[column].min(),
        vmax=gdf_merged[column].max()
    )

    # initialize figure
    (figure, axis) = pyplot.subplots(figsize=(14, 7))

    # colorbar
    colorbar = figure.colorbar(
        cm.ScalarMappable(norm=normalize, cmap=colormap),
        ax=axis
    )
    colorbar.set_label(
        "Summer land surface temperature 2002-2008 [°F]",
        rotation=270,
        labelpad=10
    )

    # main plot
    gdf_merged.plot(ax=axis, column=column, cmap=colormap, legend=False)

    # create extra room on the right for the inset
    xlim = axis.get_xlim()
    axis.set_xlim([xlim[0], xlim[1] + 2e4])

    # inset plot
    inset = axis.inset_axes(inset_position)
    gdf_plot = geopandas.GeoDataFrame(street).T
    gdf_plot.crs = gdf_roads.crs
    gdf_plot.plot(
        ax=inset, color=colormap(normalize(street[column])),
        linewidth=inset_linewidth, zorder=2
    )
    inset.margins(x=margin, y=margin)

    # roads for which we have no data. we need to disable autoscale first because
    # these functions contain data far from the road in question
    inset.autoscale(False)
    gdf_merged.plot(
        ax=inset, column=column, cmap=colormap,
        alpha=0.3, linewidth=inset_linewidth, zorder=1
    )
    gdf_roads_nodata.plot(
        ax=inset, color="black", linestyle="--",
        alpha=0.3, linewidth=4, zorder=1, label="no data"
    )
    inset.legend()

    # set inset to be equal ratio (other plotting functions can change this, so we
    # run it after all plotting functions that interact with the inset). we also
    # need to update the xlim and ylim at this point
    make_square(inset)
    inset_xlim = inset.get_xlim()
    inset_ylim = inset.get_ylim()

    # rectangle showing the location of the inset
    # transformer credit: https://stackoverflow.com/a/40475221
    axis_to_data = axis.transAxes + axis.transData.inverted()
    inset_coords_bottomleft = (inset_xlim[0], inset_ylim[0])
    inset_coords_topleft = (inset_xlim[0], inset_ylim[1])
    inset_pos_bottomleft = axis_to_data.transform(
        (inset_position[0] + inset_adjust, inset_position[1])
    )
    inset_pos_topleft = axis_to_data.transform(
        (inset_position[0] + inset_adjust, inset_position[1] + inset_position[3])
    )
    gdf_inset_location = geopandas.GeoDataFrame([{
        "geometry": shapely.geometry.Polygon([
            inset_coords_bottomleft,
            inset_coords_topleft,
            inset_pos_topleft,
            inset_pos_bottomleft
        ])
    }])
    gdf_inset_location.plot(ax=axis, color="black", zorder=3, alpha=0.5)

    # remove all coordinate ticks
    axis.set_xticks([])
    axis.set_yticks([])
    inset.set_xticks([])
    inset.set_yticks([])

    # text
    axis.text(
        -7892500, 5220000,
        "The average land surface temperature for",
        fontsize=12,
        horizontalalignment="center"
    )
    axis.text(
        -7892500, 5218400,
        "{house_numbers} {street_name}".format(
            house_numbers=address_range_from_tlid(street["TLID"]),
            street_name=street["FULLNAM"]
        ),
        fontsize=22,
        horizontalalignment="center"
    )
    axis.text(
        -7892500, 5217000,
        "was",
        fontsize=15,
        horizontalalignment="center"
    )
    axis.text(
        -7892500, 5214500,
        "{:.1f}°F".format(street["LST_weighted"]),
        fontsize=40,
        horizontalalignment="center"
    )
    axis.text(
        -7892500, 5213000,
        "in the summers from 2002-2008",
        fontsize=15,
        horizontalalignment="center"
    )

    axis.add_artist(anchored_artists.AnchoredSizeBar(
        axis.transData,
        20, "20 m", "lower left",
        pad=0.1,
        color="black",
        frameon=False,
        size_vertical=1
    ))

    # basemaps
    contextily.add_basemap(ax=axis, source=basemap)
    contextily.add_basemap(ax=inset, source=inset_basemap)

    # the above basemap code may have resized the inset
    inset.set_xlim(inset_xlim)

    pyplot.tight_layout()

    return (figure, axis)

n_to_sample = 3
for (index, street) in tqdm.tqdm(
    gdf_merged.sample(n_to_sample).iterrows(),
    total=n_to_sample,
    desc="rendering"
):
    output_file = "sample/{}.png".format(street["TLID"])    
    if os.path.isfile(output_file):
        continue

    (figure, axis) = generate_image(street)
    
    # note that the inset position is slightly off in Jupyter; it appears as
    # intended on the exported image
    pyplot.savefig(output_file)
    pyplot.close()